## 1.
Даны значения величины заработной платы заемщиков банка (zp) и значения  
их поведенческого кредитного скоринга (ks):  
zp = [35, 45, 190, 200, 40, 70, 54, 150, 120, 110],  
ks = [401, 574, 874, 919, 459, 739, 653, 902, 746, 832].  
Найдите ковариацию этих двух величин с помощью элементарных действий, а  
затем с помощью функции cov из numpy  
Полученные значения должны быть равны.  
Найдите коэффициент корреляции Пирсона с помощью ковариации и  
среднеквадратичных отклонений двух признаков, а затем с использованием  
функций из библиотек numpy и pandas.


In [1]:
import numpy as np
import scipy.stats as stats
from statsmodels.stats.weightstats import _tconfint_generic as t_st
from statsmodels.stats.weightstats import _zconfint_generic as z_st

Найдем коварацию 2 св  
Коварация = матожидание от произведения двух СВ минус произведение матожидания СВ

In [43]:
zp = np.array([35, 45, 190, 200, 40, 70, 54, 150, 120, 110])
ks = np.array([401, 574, 874, 919, 459, 739, 653, 902, 746, 832])
zp_mean = zp.mean() # мат ожидание 1 св
ks_mean = ks.mean() # мат ожидание 2 св
m_zpks = (zp*ks).mean() # матожидание произведения 1 св на др св
cov = m_zpks - zp_mean*ks_mean # матожидание произведения двух св минус произведение матожидания 1св на др
print(round(cov,2))

9157.84


Найдем коварацию через встроеную в np функцию

In [34]:
np.cov(zp,ks, ddof=0)[1,0] # возврашает кв матрицу ковариации двух соответствующих выборок 
# Cij=cov(xi,xj).  элементы главной диагонали данной матрицы Cii
# будут представлять собой дисперсии соответствующей выборки xi, а эл побочной диагонали коварацию

9157.84

Докажем что элементы главной диагонали ковариционной матрицы будут представлять собой дисперсии соответствующей св, а эл побочной диагонали коварацию велечин

In [22]:
np.cov(zp,ks, ddof=0)

array([[ 3494.64,  9157.84],
       [ 9157.84, 30468.89]])

In [41]:
print(f'дисперсия zp - {round(zp.std()**2,2)}, дисперсия ks- {round(ks.std()**2,2)}')

дисперсия zp - 3494.64, дисперсия ks- 30468.89


Найдем коэфицент кореляции пирсона
коэфицент кореляции пирсона - матожидание от произведения двух СВ минус произведение матожидания СВ деленое на произведение дисперсии этих 2 СВ

In [57]:
cor_pearson = cov/(zp.std()*ks.std())
round(cor_pirson,4)

0.8875

In [58]:
cor_pearson2 = ((zp*ks).mean()-zp.mean()*ks.mean())/(zp.std()*ks.std())
round(cor_pearson2,4)

0.8875

In [59]:
round(np.corrcoef(zp,ks)[0,1],4)

0.8875

In [60]:
np.corrcoef(zp,ks) # возвращается кореляционная матрица по гд 1 по побоччной коэф кореляции

array([[1.        , 0.88749009],
       [0.88749009, 1.        ]])

## 2.
 Измерены значения IQ выборки студентов, 
обучающихся в местных технических вузах:  
131, 125, 115, 122, 131, 115, 107, 99, 125, 111.  
Известно, что в генеральной совокупности IQ распределен нормально.  
Найдите доверительный интервал для математического ожидания с  
надежностью 0.95.


Поскольку нам не известно ст отклонение  ГС и  ГС не было проанализировано то наш критерий расчета дов интервала критерий стьюдента

In [82]:
iq = np.array([131, 125, 115, 122, 131, 115, 107, 99, 125, 111])
iq_std_error = iq.std()/(len(iq)**0.5)
lower, upper = t_st(iq.mean(), iq_std_error, len(iq)-1, 0.05, 'two-sided')
print('При задоном p уровне 0.95 доверительный интервал - от ',int(round(lower,0)),'до ', int(round(upper,0))) # доверительный интервал в статистеке округляют до точности входных данных

При задоном p уровне 0.95 доверительный интервал - от  111 до  125


In [ ]:
Более верный расчет с расчетом несмещеной десперсии т.к выборка меньше 100 и работаем мы с коэфицентом стьюдента

In [92]:
sq_list = [(el-iq.mean())**2 for el in iq]
variance_unbased = sum(sq_list)/(len(iq)-1) # несмещеная дисперсия
std_unbaiased = np.sqrt(variance_unbased)
iq_std_error = std_unbaiased/(len(iq)**0.5)
lower, upper = t_st(iq.mean(), iq_std_error, len(iq)-1, 0.05, 'two-sided')
print('При задоном p уровне 0.95 доверительный интервал - от ',int(round(lower,0)),'до ', int(round(upper,0))) # доверительный интервал в статистеке округляют до точности входных данных

При задоном p уровне 0.95 доверительный интервал - от  111 до  126


In [96]:
def conf_interval_t(var, p=0.95): # var - np массив
    from statsmodels.stats.weightstats import _tconfint_generic as t_st
    sq_list = [(el-var.mean())**2 for el in var]
    variance_unbased = sum(sq_list)/(len(var)-1) # несмещеная дисперсия
    std_unbaiased = np.sqrt(variance_unbased)
    std_error = std_unbaiased/(len(var)**0.5)
    lower, upper = t_st(var.mean(), std_error, len(var)-1, 1-p, 'two-sided')
    print('При задоном p уровне 0.95 доверительный интервал - от ',int(round(lower,0)),'до ', int(round(upper,0))) # доверительный интервал в статистеке округляют до точности входных данных

In [97]:
a = conf_interval_t(var=np.array([131, 125, 115, 122, 131, 115, 107, 99, 125, 111]))
a

При задоном p уровне 0.95 доверительный интервал - от  111 до  126


## 3.
Известно, что рост футболистов в сборной распределен нормально с  
дисперсией генеральной совокупности, равной 25 кв.см. Объем выборки равен  
27, среднее выборочное составляет 174.2. Найдите доверительный интервал  
для математического ожидания с надежностью 0.95.

Известно ст откланение следовательно гс была проанализирована а занчит наш критерий  
для поиска дов интервала Z критерий фишера

In [100]:
X_med_v = 174.2
sigma_gs = 5
n=27
lower, upper = z_st(X_med_v, sigma_gs/np.sqrt(n),0.05,'two-sided')
print('При задоном p уровне 0.95 доверительный интервал - от ',int(round(lower,0)),'до ', int(round(upper,0))) # доверительный интервал в статистеке округляют до точности входных 

При задоном p уровне 0.95 доверительный интервал - от  172 до  176


In [98]:
def conf_interval_z(X_med_v, sigma_gs, n, p=0.95): # var - np массив
    from statsmodels.stats.weightstats import _zconfint_generic as z_st
    lower, upper = z_st(X_med_v, sigma_gs/np.sqrt(n),1-p,'two-sided')
    print('При задоном p уровне 0.95 доверительный интервал - от ',int(round(lower,0)),'до ', int(round(upper,0))) # доверительный интервал в статистеке округляют до точности входных данных

In [99]:
conf_interval_z(174.2, 5, 27)

При задоном p уровне 0.95 доверительный интервал - от  172 до  176
